In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("clustering_gmm_hw.ipynb")

# Homework: Hierarchical Clustering and Gaussian Mixture Models

## Instructions

This homework will help you practice hierarchical clustering and Gaussian Mixture Models using scikit-learn. Complete all questions and run all cells. 


In [ ]:
# Run this cell to import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs, make_moons
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
# import otter

# grader = otter.Notebook("clustering_gmm_hw.ipynb")

# Set random seed for reproducibility
np.random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

---
## Part 1: Hierarchical Clustering (20 points)

In this section, you'll work with wine quality data to perform hierarchical clustering.

You'll be using scikit-learn's [hierarchical clustering](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering).

In [ ]:
# Generate synthetic wine quality dataset
np.random.seed(42)
n_samples = 150

# Create three types of wine with different characteristics
# Type 1: High alcohol, low acidity
wine1 = np.random.multivariate_normal(
    mean=[13.5, 3.2, 2.8], 
    cov=[[0.3, 0.05, 0.02], [0.05, 0.2, 0.01], [0.02, 0.01, 0.15]], 
    size=50
)

# Type 2: Medium alcohol, high acidity
wine2 = np.random.multivariate_normal(
    mean=[11.5, 5.5, 2.2], 
    cov=[[0.25, -0.05, 0.03], [-0.05, 0.3, 0.02], [0.03, 0.02, 0.12]], 
    size=50
)

# Type 3: Low alcohol, medium acidity
wine3 = np.random.multivariate_normal(
    mean=[9.5, 4.2, 3.5], 
    cov=[[0.2, 0.03, -0.02], [0.03, 0.25, 0.01], [-0.02, 0.01, 0.2]], 
    size=50
)

wine_data = np.vstack([wine1, wine2, wine3])
true_labels = np.array([0]*50 + [1]*50 + [2]*50)

wine_df = pd.DataFrame(wine_data, columns=['alcohol', 'acidity', 'pH'])
wine_df['true_type'] = true_labels

print("Wine Dataset Shape:", wine_df.shape)
print("\nFirst few rows:")
print(wine_df.head())

# Visualize the data
fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(131)
ax1.scatter(wine_df['alcohol'], wine_df['acidity'], c=wine_df['true_type'], cmap='viridis', alpha=0.6)
ax1.set_xlabel('Alcohol')
ax1.set_ylabel('Acidity')
ax1.set_title('Alcohol vs Acidity')

ax2 = fig.add_subplot(132)
ax2.scatter(wine_df['alcohol'], wine_df['pH'], c=wine_df['true_type'], cmap='viridis', alpha=0.6)
ax2.set_xlabel('Alcohol')
ax2.set_ylabel('pH')
ax2.set_title('Alcohol vs pH')

ax3 = fig.add_subplot(133)
ax3.scatter(wine_df['acidity'], wine_df['pH'], c=wine_df['true_type'], cmap='viridis', alpha=0.6)
ax3.set_xlabel('Acidity')
ax3.set_ylabel('pH')
ax3.set_title('Acidity vs pH')

plt.tight_layout()
plt.show()

### Question 1.1: Computing Linkage Matrix (5 points)

Compute the linkage matrix for the wine data using **Ward's method**. Store the result in a variable called `linkage_ward`. Use only the feature columns (alcohol, acidity, pH) - not the true_type column.

In [ ]:
X_wine = wine_df[['alcohol', 'acidity', 'pH']].values

linkage_ward = ...

In [ ]:
grader.check("q1_1")

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1.2: Visualizing the Dendrogram (5 points)

Create a dendrogram from your linkage matrix. Make sure to:
- Add appropriate title and labels
- Use `truncate_mode='lastp'` with `p=30` to make it readable

In [ ]:
plt.figure(figsize=(12, 6))

# YOUR CODE HERE
...

plt.show()


<!-- END QUESTION -->

### Question 1.3: Comparing Linkage Methods (5 points)

Compare three different linkage methods (single, complete, and average) by computing the silhouette score for each when cutting the dendrogram into 3 clusters. Store your results in variables:
- `silhouette_single`
- `silhouette_complete`
- `silhouette_average`

In [ ]:
# YOUR CODE HERE
# Compute linkage for each method and get cluster labels

linkage_single = ...
linkage_complete = ...
linkage_average = ...

# Get cluster labels for k=3
labels_single = ...
labels_complete = ...
labels_average = ...

# Compute silhouette scores
silhouette_single = ...
silhouette_complete = ...
silhouette_average = ...


print(f"Silhouette Score (Single): {silhouette_single:.4f}")
print(f"Silhouette Score (Complete): {silhouette_complete:.4f}")
print(f"Silhouette Score (Average): {silhouette_average:.4f}")


In [ ]:
grader.check("q1_3")

### Question 1.4: Optimal Number of Clusters (5 points)

Using Ward's linkage, determine the optimal number of clusters by computing silhouette scores for k=2 to k=10. Store the silhouette scores in an array called `silhouette_scores` and the optimal number of clusters in `optimal_k`.

In [ ]:
# YOUR CODE HERE
k_range = range(2, 11)
silhouette_scores = []

for k in k_range:

    labels = ...
    score = ...

    silhouette_scores.append(score)

silhouette_scores = np.array(silhouette_scores)

optimal_k = ...


# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(k_range, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.axvline(x=optimal_k, color='r', linestyle='--', label=f'Optimal k={optimal_k}')
plt.legend()
plt.grid(True)
plt.show()

print(f"Optimal number of clusters: {optimal_k}")


In [ ]:
grader.check("q1_4")

---
## Part 2: Gaussian Mixture Models (20 points)

In this section, you'll work with customer segmentation data to perform GMM clustering.

You'll be using scikit-learn's [Gaussian Mixture Models](https://scikit-learn.org/stable/modules/mixture.html).  See also the [API documentation](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture).

In [ ]:
# Generate synthetic customer data
np.random.seed(42)

# Customer Segment 1: Budget shoppers (low spending, high frequency)
segment1 = np.random.multivariate_normal(
    mean=[30, 15], 
    cov=[[50, 10], [10, 20]], 
    size=100
)

# Customer Segment 2: Regular shoppers (medium spending, medium frequency)
segment2 = np.random.multivariate_normal(
    mean=[70, 10], 
    cov=[[100, -20], [-20, 15]], 
    size=150
)

# Customer Segment 3: Premium shoppers (high spending, low frequency)
segment3 = np.random.multivariate_normal(
    mean=[120, 6], 
    cov=[[150, -10], [-10, 10]], 
    size=80
)

customer_data = np.vstack([segment1, segment2, segment3])
true_segments = np.array([0]*100 + [1]*150 + [2]*80)

customer_df = pd.DataFrame(customer_data, columns=['spending', 'frequency'])
customer_df['true_segment'] = true_segments

print("Customer Dataset Shape:", customer_df.shape)
print("\nFirst few rows:")
print(customer_df.head())

# Visualize the data
plt.figure(figsize=(10, 6))
plt.scatter(customer_df['spending'], customer_df['frequency'], 
            c=customer_df['true_segment'], cmap='viridis', alpha=0.6, s=50)
plt.xlabel('Average Spending ($)')
plt.ylabel('Visit Frequency (per month)')
plt.title('Customer Segments')
plt.colorbar(label='True Segment')
plt.show()

### Question 2.1: Fitting a GMM (5 points)

Fit a Gaussian Mixture Model with 3 components to the customer data. Store the fitted model in `gmm_model` and the predicted labels in `gmm_labels`. Use:
- `n_components=3`
- `covariance_type='full'`
- `random_state=42`

In [ ]:
X_customer = customer_df[['spending', 'frequency']].values

# YOUR CODE HERE
...

print(f"Converged: {gmm_model.converged_}")
print(f"Number of iterations: {gmm_model.n_iter_}")


In [ ]:
grader.check("q2_1")

<!-- BEGIN QUESTION -->

### Question 2.2: Visualizing GMM Results (5 points)

Create a scatter plot showing the GMM cluster assignments. Include:
- Points colored by predicted cluster
- Cluster centers marked with 'X'
- Appropriate labels and title

In [ ]:
plt.figure(figsize=(10, 6))

# YOUR CODE HERE
...

plt.show()


<!-- END QUESTION -->

### Question 2.3: Model Selection with BIC (5 points)

Compare GMMs with different numbers of components (1 to 8) using the Bayesian Information Criterion (BIC). Store the BIC values in an array called `bic_scores` and the optimal number of components in `optimal_components`.

See also the [API documentation](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture.bic).

In [ ]:
# YOUR CODE HERE
n_components_range = range(1, 9)
bic_scores = []

for n in n_components_range:

    gmm = ...
    ...
    bic = ...

    bic_scores.append(bic)

bic_scores = np.array(bic_scores)

optimal_components = ...


# Plot BIC scores
plt.figure(figsize=(10, 5))
plt.plot(n_components_range, bic_scores, 'ro-')
plt.xlabel('Number of Components')
plt.ylabel('BIC Score')
plt.title('BIC Score vs Number of Components')
plt.axvline(x=optimal_components, color='g', linestyle='--', 
            label=f'Optimal n={optimal_components}')
plt.legend()
plt.grid(True)
plt.show()

print(f"Optimal number of components: {optimal_components}")


In [ ]:
grader.check("q2_3")

### Question 2.4: Comparing Covariance Types (5 points)

Compare GMMs with 3 components using different covariance types ('full', 'tied', 'diag', 'spherical'). Compute the BIC for each and store the results in a dictionary called `covariance_bic` with keys as covariance types and values as BIC scores.

In [ ]:
# YOUR CODE HERE
covariance_types = ['full', 'tied', 'diag', 'spherical']
covariance_bic = {}

for cov_type in covariance_types:

    gmm = ...
    ...
    covariance_bic[cov_type] = ...


# Plot comparison
plt.figure(figsize=(10, 5))
plt.bar(covariance_bic.keys(), covariance_bic.values())
plt.xlabel('Covariance Type')
plt.ylabel('BIC Score')
plt.title('BIC Score by Covariance Type (3 components)')
plt.grid(axis='y')
plt.show()

print("BIC Scores by Covariance Type:")
for cov_type, bic in covariance_bic.items():
    print(f"{cov_type}: {bic:.2f}")


In [ ]:
grader.check("q2_4")

---
## Question 3: GMM vs K-means (5 points)

Compare GMM and K-means clustering on non-spherical data. Compute the Adjusted Rand Index (ARI) for both methods and store them in `ari_gmm` and `ari_kmeans`.

In [ ]:
# Number of samples of larger component
n_samples = 1000

# C is a transfomation that will make a heavily skewed 2-D Gaussian
C = np.array([[0.1, -0.1], [1.7, .8]])

import warnings
warnings.filterwarnings('ignore')

rng = np.random.default_rng(0)

# now we construct a data matrix that has n_samples from the skewed distribution,
# and n_samples/2 from a symmetric distribution offset to position (-3, 1)
X = np.r_[(rng.standard_normal((n_samples, 2)) @ C),
          .7 * rng.standard_normal((n_samples//2, 2)) + np.array([-2, 1])]
y = np.array([0] * n_samples + [1] * (n_samples//2))

plt.scatter(X[:, 0], X[:, 1], s = 10, alpha = 0.8, c=y, cmap='viridis')
plt.axis('equal')
plt.axis('off')
plt.show()

In [ ]:
# YOUR CODE HERE
from sklearn.cluster import KMeans

# Fit GMM

gmm_moons = ...
...
labels_gmm = ...

# Fit K-means
kmeans_moons = ...
...
labels_kmeans = ...

# Compute ARI
ari_gmm = ...
ari_kmeans = ...


print(f"GMM ARI: {ari_gmm:.4f}")
print(f"K-means ARI: {ari_kmeans:.4f}")

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(X[:, 0], X[:, 1], c=labels_gmm, cmap='viridis', alpha=0.6)
axes[0].set_title(f'GMM (ARI={ari_gmm:.3f})')
axes[1].scatter(X[:, 0], X[:, 1], c=labels_kmeans, cmap='viridis', alpha=0.6)
axes[1].set_title(f'K-means (ARI={ari_kmeans:.3f})')
plt.tight_layout()
plt.show()


In [ ]:
grader.check("q3")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Execute all cells and save the notebook before submitting.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)